## Imports

In [1]:
import pandas as pd
from ens_load_forecast.paths import PATH_LOAD_ACTUAL, PATH_WEATHER, PATH_LOAD_FORECAST, PATH_ZONES_AND_STATIONS, PATH_MAP_DATA, PATH_DATA
from ens_load_forecast.graphs import plot_load_per_zone, plot_load_seasonal, plot_on_map
from ens_load_forecast.data_preprocessing import get_load_forecast, get_load_actual, get_weather,get_preprocessed_weather, get_merged_dataset
import ens_load_forecast.constants as cst
import numpy as np


## Actual load

In [2]:
df_load_actual = get_load_actual()
# plot_load_per_zone(df = df_load_actual)

In [3]:
# plot_load_seasonal(df = df_load_actual, zone="N.Y.C.")

In [4]:
# df = df_load_actual.groupby("zone").apply(lambda x:x.mean())
# df["zone"] = df.index
# plot_on_map(df=df, quantity_key="load")

## Load forecast

In [5]:
df_load_forecast = get_load_forecast()
# plot_load_per_zone(df = df_load_forecast[df_load_forecast[cst.FORECAST_HORIZON] == 2])

/home/enisol/projects/ens_load_forecast/ens_load_forecast/data_preprocessing.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tmp_col"] = df.index


In [6]:
# df_load_forecast[cst.FORECAST_HORIZON].unique()

In [7]:
# plot_load_seasonal(df = df_load_forecast[df_load_forecast[cst.FORECAST_HORIZON] == 2], zone="NYISO")

## Weather

In [8]:
df_weather = get_weather(force_recompute=False)

In [9]:
df_weather

,zone,tmp,dpt,sky,wdr,gst,psn
delivery_ts,,,,,,,
2020-06-29 23:00:00-05:00,CAPITL,66.44,62.76,67.13,34.93,8.21,0.0
2020-06-29 23:00:00-05:00,CENTRL,64.94,62.39,44.74,30.67,4.56,0.0
2020-06-29 23:00:00-05:00,DUNWOD,69.00,64.00,31.00,34.00,7.00,0.0
2020-06-29 23:00:00-05:00,GENESE,67.60,62.15,19.25,7.55,6.55,0.0
2020-06-29 23:00:00-05:00,HUD VL,66.68,63.04,42.15,34.56,4.89,0.0
...,...,...,...,...,...,...,...
2023-06-29 23:00:00-05:00,MHK VL,61.82,56.43,10.13,14.32,3.89,0.0
2023-06-29 23:00:00-05:00,MILLWD,68.00,62.00,20.00,19.00,5.00,0.0
2023-06-29 23:00:00-05:00,N.Y.C.,73.95,61.63,20.16,18.42,7.21,0.0


## Merge data

In [10]:
df_merged = get_merged_dataset(df_load_actual=df_load_actual, df_load_forecast=df_load_forecast, df_weather=df_weather)

In [11]:
from ens_load_forecast.features_engineering import add_day_of_week, process_wind_direction
df_features = process_wind_direction(add_day_of_week(df_merged))

In [12]:

# series = df_merged.groupby(cst.ZONE).apply(lambda x: np.sqrt(((x["load"] - x["load_forecast"])**2).mean()))
# df = pd.DataFrame(data=series, index=series.index, columns=["load_forecast_error"])
# df["zone"] = df.index
# plot_on_map(df = df, quantity_key="load_forecast_error")

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

models = {}
mse = {}
for zone in df_features[cst.ZONE].unique():
    df = df_features[df_features[cst.ZONE] == zone]
    # model = Pipeline(
    #     steps=[
    #         ("polynomial_features", PolynomialFeatures(degree=2, include_bias=False)),
    #         ("linear_regression", LinearRegression(fit_intercept=True)),
    #     ]
    # )
    model = GradientBoostingRegressor() 
    model.fit(X = df[cst.FEATURES_LIST], y = df["load"])
    models[zone] = model
    mse[zone] = np.sqrt(((model.predict(X=df[cst.FEATURES_LIST]) - df["load"])**2).mean())

In [15]:
df_mse = pd.DataFrame(data=pd.Series(mse), columns=["mse"])
df_mse

,mse
CAPITL,50.910383
CENTRL,55.881165
DUNWOD,28.631655
GENESE,37.730104
HUD VL,44.670012
LONGIL,104.639295
MHK VL,42.047039
MILLWD,21.110713
N.Y.C.,140.524255
NORTH,22.140644


In [ ]:

series = df_merged.groupby(cst.ZONE).apply(lambda x: np.sqrt(((x["load"] - x["load_forecast"])**2).mean()))
df_original_mse = pd.DataFrame(data=series, index=series.index, columns=["load_forecast_error"])
df_original_mse["zone"] = df_original_mse.index

In [ ]:
df = pd.DataFrame(df_mse["mse"] / df_mse["original_mse"], columns=["mse_normalized"])
df[cst.ZONE] = df.index
plot_on_map(df=df, quantity_key="mse_normalized")

In [ ]:
df = df_merged[df_merged[cst.ZONE] == "N.Y.C."]


(df["load_forecast"] - df["load"]).plot(backend="plotly")